In [ ]:
import numpy as np

# --- 1. Setup & Data Generation ---
np.random.seed(42)

# Feature = lab test score 
# Healthy Patients = low score (approx 20)
# Sick patients = high score (approx 60)
X_healthy = np.random.normal(loc=20, scale=5, size=(99, 1))
X_sick = np.random.normal(loc=60, scale=5, size=(1, 1))

X = np.vstack([X_healthy, X_sick])
y = np.array([0]*99 + [1]).reshape(-1, 1)

# --- 2. Preprocessing (The Correct Way) ---
# CRITICAL FIX: Calculate stats *once* and store them. 
# You need these keys to unlock future predictions.
train_mean = np.mean(X)
train_std = np.std(X)

# Normalize the training data
X_scaled = (X - train_mean) / train_std

# --- 3. Model Definitions ---
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# --- 4. Training ---
w = np.zeros((1, 1))
b = 0.0
learning_rate = 0.01
epochs = 10000
m = X.shape[0]
clip_threshold = 1.0

print("Training model...")
for epoch in range(epochs):
    # Forward pass
    z = np.dot(X_scaled, w) + b
    y_hat = sigmoid(z)
    
    # Loss (Binary Cross Entropy)
    eps = 1e-15
    y_hat_clipped = np.clip(y_hat, eps, 1-eps)
    loss = -np.mean(y * np.log(y_hat_clipped) + (1-y) * np.log(1 - y_hat_clipped))
    
    # Backward pass
    dz = y_hat - y
    dw = (1/m) * np.dot(X_scaled.T, dz)
    db = (1/m) * np.sum(dz)
    
    # Gradient clipping
    dw = np.clip(dw, -clip_threshold, clip_threshold)
    db = np.clip(db, -clip_threshold, clip_threshold)
    
    # Update
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    if epoch % 2000 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# --- 5. Evaluation ---
def evaluate_model(y_true, y_pred_probs, threshold=0.3):
    y_pred = (y_pred_probs >= threshold).astype(int)
    
    TP = np.sum((y_pred == 1) & (y_true == 1))
    TN = np.sum((y_pred == 0) & (y_true == 0))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    FN = np.sum((y_pred == 0) & (y_true == 1))
    
    return TP, TN, FP, FN

def calculate_metrics(TP, TN, FP, FN):
    # Avoid division by zero
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

# Get predictions on training set
z_final = np.dot(X_scaled, w) + b
y_pred_probs = sigmoid(z_final)

# Calculate Counts
TP, TN, FP, FN = evaluate_model(y, y_pred_probs, threshold=0.3)

# CRITICAL FIX: Unpack into exactly 3 variables
precision, recall, f1 = calculate_metrics(TP, TN, FP, FN)

print("\n--- RESULTS ---")
print("Confusion Matrix:")
print(f"TP: {TP} | FP: {FP}")
print(f"FN: {FN} | TN: {TN}")
print("-" * 15)
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1 Score:  {f1:.2f}")

# --- 6. Inference (Real World Test) ---
def predict_new_patient(score_raw, w, b, t_mean, t_std, threshold=0.3):
    """
    Predicts status for a new patient using raw lab scores.
    CRITICAL: Must use the training set's mean/std to normalize.
    """
    # 1. Normalize
    score_scaled = (score_raw - t_mean) / t_std
    
    # 2. Predict
    prob = sigmoid(score_scaled * w + b)
    is_sick = prob >= threshold
    
    return prob[0][0], is_sick[0][0]

print("\n--- INFERENCE TESTS ---")
# Test 1: Low score (should be healthy)
score_low = 20
prob, is_sick = predict_new_patient(score_low, w, b, train_mean, train_std)
print(f"Patient Score {score_low}: {'SICK' if is_sick else 'HEALTHY'} (Prob: {prob:.4f})")

# Test 2: High score (should be sick)
score_high = 60
prob, is_sick = predict_new_patient(score_high, w, b, train_mean, train_std)
print(f"Patient Score {score_high}: {'SICK' if is_sick else 'HEALTHY'} (Prob: {prob:.4f})")

Training model...
Epoch 0, Loss: 0.6931
Epoch 2000, Loss: 0.0628
Epoch 4000, Loss: 0.0328
Epoch 6000, Loss: 0.0224
Epoch 8000, Loss: 0.0171

--- RESULTS ---
Confusion Matrix:
TP: 1 | FP: 0
FN: 0 | TN: 99
---------------
Precision: 1.00
Recall:    1.00
F1 Score:  1.00

--- INFERENCE TESTS ---
Patient Score 20: HEALTHY (Prob: 0.0104)
Patient Score 60: SICK (Prob: 0.8891)
